In [ ]:
use database research;
use schema aagarwal;

In [ ]:
-- Create a table for our documents
CREATE OR REPLACE TABLE product_documents (
    product_id NUMBER,
    product_name VARCHAR,
    category VARCHAR,
    description VARCHAR,
    price NUMBER(10,2),
    created_date DATE
);

In [ ]:
INSERT INTO product_documents (product_id, product_name, category, description, price, created_date)
VALUES
    (1, 'Ultra Comfort Office Chair', 'Furniture', 'Ergonomic office chair with lumbar support and adjustable height. Features breathable mesh back and padded armrests for all-day comfort during long work sessions.', 299.99, '2023-01-15'),
    (2, 'Professional Laptop Stand', 'Accessories', 'Aluminum laptop stand with adjustable height and angle settings. Improves posture and prevents neck strain. Compatible with laptops up to 17 inches.', 79.99, '2023-02-20'),
    (3, 'Smart LED Desk Lamp', 'Lighting', 'Dimmable LED desk lamp with 5 color modes and 7 brightness levels. Features USB charging port and touch controls. Energy-efficient with auto-off timer.', 49.99, '2023-03-10'),
    (4, 'Wireless Noise-Cancelling Headphones', 'Electronics', 'Over-ear headphones with active noise cancellation. 30-hour battery life, Bluetooth 5.0 connectivity, and built-in microphone for calls. Includes carrying case and audio cable.', 199.99, '2023-04-05'),
    (5, 'Adjustable Standing Desk', 'Furniture', 'Electric standing desk with memory presets for height adjustments. Smooth and quiet motor operation. Spacious surface that supports up to 200 pounds.', 449.99, '2023-05-12'),
    (6, 'Mechanical Keyboard', 'Electronics', 'RGB backlit mechanical keyboard with customizable key switches. Features programmable macros, anti-ghosting, and durable aluminum frame. Ergonomic design for comfortable typing.', 129.99, '2023-06-18'),
    (7, 'Ultrawide Curved Monitor', 'Electronics', '34-inch ultrawide curved monitor with 3440x1440 resolution. Features HDR support, 144Hz refresh rate, and adaptive sync technology for smooth visuals. Built-in speakers and multiple connectivity options.', 599.99, '2023-07-25'),
    (8, 'Ergonomic Mouse', 'Accessories', 'Vertical ergonomic mouse designed to reduce wrist strain. Adjustable DPI settings, programmable buttons, and long-lasting battery. Compatible with Windows and macOS.', 59.99, '2023-08-30'),
    (9, 'Cable Management System', 'Accessories', 'Complete desk cable management kit including clips, sleeves, and ties. Keeps workspace organized and prevents cable tangling. Easy installation with adhesive backing.', 24.99, '2023-09-14'),
    (10, 'Air Purifier', 'Appliances', 'HEPA air purifier for office spaces up to 500 square feet. Removes 99.97% of allergens, dust, and odors. Features quiet operation, auto mode, and filter replacement indicator.', 169.99, '2023-10-05');

In [ ]:
-- Create a table to store the embeddings
CREATE OR REPLACE TABLE product_embeddings (
    product_id number,
    description_embedding vector(Float, 768)
)

In [ ]:
-- Generate embeddings using FLATTEN for each product description
INSERT INTO product_embeddings (product_id, description_embedding)
SELECT 
    product_id,
    SNOWFLAKE.CORTEX.EMBED_TEXT_768('snowflake-arctic-embed-m-v1.5', description)
FROM 
    product_documents;

Table product_embeddings;

In [ ]:
# Import necessary libraries
from snowflake.snowpark.functions import col, lit
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

def retrieve_relevant_documents(query_text, top_k=3):
    """
    Retrieve the most relevant documents based on vector similarity.
    
    Args:
        query_text: The user's query text
        top_k: Number of results to return
        
    Returns:
        DataFrame of relevant documents with similarity scores
    """
    
    # Generate embedding for the query  
    query_result = session.sql( """
            SELECT SNOWFLAKE.CORTEX.EMBED_TEXT_768('snowflake-arctic-embed-m-v1.5', ?) AS query_embedding
            """,
            [query_text]
    )

    rows = query_result.collect()
    query_embedding = rows[0]["QUERY_EMBEDDING"]
    
    # Find the most relevant documents using vector similarity
    results = session.sql(f"""
        SELECT 
            p.product_id,
            p.product_name,
            p.category,
            p.description,
            p.price,
            VECTOR_COSINE_SIMILARITY(e.description_embedding, {query_embedding}::VECTOR(FLOAT, 768)) as similarity
        FROM 
            product_documents p
        JOIN 
            product_embeddings e ON p.product_id = e.product_id
        ORDER BY 
            similarity DESC
        LIMIT {top_k}
    """).to_pandas()

    return results

# Test the retrieval function
test_query = "What products help with back pain?"
relevant_docs = retrieve_relevant_documents(test_query)
print(f"Query: {test_query}\n")
print("Top relevant products:")
for idx, row in relevant_docs.iterrows():
    print(f"{idx+1}. {row['PRODUCT_NAME']} (Similarity: {row['SIMILARITY']:.4f})")
    print(f"   Category: {row['CATEGORY']}")
    print(f"   Price: ${row['PRICE']}")
    print(f"   Description: {row['DESCRIPTION'][:100]}...\n")

In [ ]:
def rag_product_assistant(query_text):
    """
    RAG-based product assistant that answers user queries.
    
    Args:
        query_text: The user's query
        
    Returns:
        Generated response based on retrieved context
    """
    # Retrieve relevant documents
    relevant_docs = retrieve_relevant_documents(query_text)
    
    # Create context from retrieved documents
    context = "Based on the following product information:\n\n"
    
    for idx, doc in relevant_docs.iterrows():
        context += f"Product {idx+1}: {doc['PRODUCT_NAME']}\n"
        context += f"Category: {doc['CATEGORY']}\n"
        context += f"Price: ${doc['PRICE']}\n"
        context += f"Description: {doc['DESCRIPTION']}\n\n"
    
    # Create prompt for the LLM
    prompt = f"""{context}
    
        User question: {query_text}
        
        Please provide a helpful answer based on the product information above. If you cannot answer the question based on the provided information, please state that clearly.
        """
    
    # Generate response using Snowflake's LLM integration
    response_result = session.sql(
            """
            SELECT SNOWFLAKE.CORTEX.COMPLETE(?, ?) AS response
            """,
            ['llama2-70b-chat',prompt]
    ).collect()
    
    response = response_result[0]['RESPONSE']
    return response, relevant_docs

# You can test the full RAG system with a sample query
sample_query = "What products would you recommend for better ergonomics?"
response, docs = rag_product_assistant(sample_query)

print(f"Query: {sample_query}\n")
print("Generated Response:")
print(response)
print("\nBased on these products:")
for idx, doc in docs.iterrows():
    print(f"{idx+1}. {doc['PRODUCT_NAME']} (Similarity: {doc['SIMILARITY']:.4f})")

In [ ]:
# Define test queries
test_queries = [
    "What products would you recommend for better ergonomics?",
    "Tell me about standing desk options",
    "What's good for reducing neck strain?",
    "Compare the noise-cancelling headphones with other electronics",
    "What's the most affordable product for cable management?",
    "What smartphone options do you have?"  # This is outside our knowledge base
]

# Test each query
for query in test_queries:
    print(f"\n\n{'='*80}\nQUERY: {query}\n{'='*80}")
    
    response, docs = rag_product_assistant(query)
    
    print("\nRESPONSE:")
    print(response)
    
    print("\nRETRIEVED PRODUCTS:")
    for idx, doc in docs.iterrows():
        print(f"{idx+1}. {doc['PRODUCT_NAME']} (Similarity: {doc['SIMILARITY']:.4f})")